In [2]:
%%timeit -r 10 -n 30

'''
Devo tradurre la query di innestata dal dialetto Oracle al python

SELECT AE.NOMEMODELLO
FROM APPARECCHIATUREELETTRONICHE AE
WHERE AE.NOMEMODELLO IN (
   SELECT AE_INNER.NOMEMODELLO
   FROM APPARECCHIATUREELETTRONICHE AE_INNER
   WHERE SYS.DBMS_CRYPTO.HASH(UTL_RAW.CAST_TO_RAW(CONCAT(AE_INNER.MANUALEUTENTE, AE_INNER.NOMEMODELLO)), 3) = SYS.DBMS_CRYPTO.HASH(UTL_RAW.CAST_TO_RAW(CONCAT(AE.MANUALEUTENTE, AE.NOMEMODELLO)), 3)
         AND DBMS_LOB.GETLENGTH(AE_INNER.MANUALEUTENTE) > 0
         AND AE_INNER.ID IN (
             SELECT PO.ID
             FROM PRODOTTIORDINATI PO
             WHERE PO.NUMEROORDINE IN (
                SELECT OC.NUMEROORDINE
                FROM ORDINICLIENTI OC
                WHERE OC.CITTASPEDIZIONE = 'MILANO'
                     AND TO_DATE(OC.DATAORDINE, 'DD-MON-RR', 'NLS_DATE_LANGUAGE=ITALIAN') BETWEEN TO_DATE('01-LUG-23', 'DD-MON-RR') AND TO_DATE('31-LUG-23', 'DD-MON-RR')
             )
         )
);

Il database utilizzato è un database OracleXE21C, commenti ben dettagliati del codice, suddivisione precisa del codice, gestisci al meglio delle eccezioni.
Utilizza la libreria CX_Oracle e suddividi tutto il codice in moduli separati tra di loro.

Fai attenzione alla sicurezza dei dati.
Voglio che l'output del codice venga strutturato in ordine temporale


#import librerie
#connessione al database
#funzione per eseguire la query
#main
#disconnessione database
'''

import cx_Oracle
import pandas as pd
import datetime

#connessione al database
def connessione():
    try:
        conn = cx_Oracle.connect('system/oirad1993@localhost:1521/XEPDB1')
        return conn
    except cx_Oracle.Error as error:
        print(error)

#disconnessione database
def disconnessione(conn):
    try:
        conn.close()
    except cx_Oracle.Error as error:
        print(error)

#funzione per eseguire la query

def esegui_query(conn, query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor
    except cx_Oracle.Error as error:
        print(error)

#main
if __name__ == '__main__':
    conn = connessione()
    query = '''SELECT AE.NOMEMODELLO
                FROM APPARECCHIATUREELETTRONICHE AE
                WHERE AE.NOMEMODELLO IN (
                    SELECT AE_INNER.NOMEMODELLO
                    FROM APPARECCHIATUREELETTRONICHE AE_INNER
                    WHERE SYS.DBMS_CRYPTO.HASH(UTL_RAW.CAST_TO_RAW(CONCAT(AE_INNER.MANUALEUTENTE, AE_INNER.NOMEMODELLO)), 3) = SYS.DBMS_CRYPTO.HASH(UTL_RAW.CAST_TO_RAW(CONCAT(AE.MANUALEUTENTE, AE.NOMEMODELLO)), 3)
                        AND DBMS_LOB.GETLENGTH(AE_INNER.MANUALEUTENTE) > 0
                        AND AE_INNER.ID IN (
                            SELECT PO.ID
                            FROM PRODOTTIORDINATI PO
                            WHERE PO.NUMEROORDINE IN (
                                SELECT OC.NUMEROORDINE
                                FROM ORDINICLIENTI OC
                                WHERE OC.CITTASPEDIZIONE = 'MILANO'
                                    AND TO_DATE(OC.DATAORDINE, 'DD-MON-RR', 'NLS_DATE_LANGUAGE=ITALIAN') BETWEEN TO_DATE('01-LUG-23', 'DD-MON-RR') AND TO_DATE('31-LUG-23', 'DD-MON-RR')
                            )
                        )
                )'''
    cursor = esegui_query(conn, query)
    df = pd.DataFrame(cursor.fetchall())
    df.columns = cursor.column_names

    #disconnessione database
    disconnessione(conn)

    #output
    print(df)




ORA-12514: TNS: il listener non è attualmente a conoscenza del servizio richiesto nel descrittore di connessione


AttributeError: 'NoneType' object has no attribute 'cursor'